Neuroevolution: Exercise 1 
=========
###### Artur Ganzha 10019651
---------	
###### Raul Gorek 10061333
---------	

### Aufgabe 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time

In [ ]:
class Dice:
    def __init__(self, no_init=False):
        if no_init:
            self.value = 0
        else:
            self.value = random.randint(1,6)
    
    def roll(self):
        self.value = random.randint(1,6)

In [ ]:
wuerfel_liste = [Dice() for _ in range(100)]

t_start = time.perf_counter()
while sum([wuerfel.value for wuerfel in wuerfel_liste]) != 600:
    for wuerfel in wuerfel_liste:
        wuerfel.roll()
    if time.perf_counter() - t_start > 180:
        break

In [ ]:
class Individual:
    def __init__(self, child=False, num=100):
        self.num = num
        if child:
            self.wuerfel_liste = np.zeros(shape=(num,))
            self.fitness = 0
        else:
            self.wuerfel_liste = np.random.randint(1, 7, size=(num,))
            self.fitness = np.count_nonzero(self.wuerfel_liste == 6)

    def mutate(self, p):
        mask = np.random.random(size=(self.num,)) <= p
        self.wuerfel_liste = np.multiply(self.wuerfel_liste, np.logical_not(mask))
        self.wuerfel_liste = np.add(
            np.multiply(np.random.randint(1, 7, size=mask.size), mask),
            self.wuerfel_liste,
        )
        self.fitness = np.count_nonzero(self.wuerfel_liste == 6)

In [ ]:
popsize = 100
selection = 20
mutrate = 1
max_gen = 1000

population = [Individual() for _ in range(popsize)]

generation = 0
while generation < max_gen:
    fittest = sorted(population, key=lambda x: x.fitness, reverse=True)
    fittest = fittest[0:selection]
    if fittest[0].fitness == 100:
        break
    if generation % (max_gen*0.1) == 0:
        mutrate = mutrate * 0.5
    print(f"gen={generation}, mutrate={mutrate}, fittest={fittest[0].fitness}")
    #print(fittest[0].fitness)
    population = fittest[:]
    while len(population) < popsize:
        random.shuffle(fittest)
        mom = fittest.pop()
        dad = fittest.pop()
        child = Individual(child=True)
        for i in range(len(mom.wuerfel_liste)):
            child.wuerfel_liste[i] = mom.wuerfel_liste[i] if random.random() < 0.5 else dad.wuerfel_liste[i]
        child.mutate(mutrate)
        population.append(child)
        fittest.append(mom)
        fittest.append(dad)

    generation += 1

fittest = sorted(population, key=lambda x: x.fitness, reverse=True)
print(f"gen={generation}, mutrate={mutrate}, fittest={fittest[0].fitness}")

Man kann zum einen die Parameter anpassen (z.B. Mutationsrate, Selektionsgröße)
Zum anderen lässt sich durch einen Epsilon-Greedy ähnlichen Abfall der Mutationsrate erreichen, dass der Algorithmus am Anfang große Menge 6en erzeugt, wo sie noch nicht vorhanden sind und am Ende diese nicht mehr löscht, wo sie bereits bestehen.

Als Laufzeitoptimierung wurde eine Darstellung der Klasse - Individual mit Numpy Array verwendet.

### Aufgabe 2

In [ ]:
import networkx as nx
import numpy as np

In [ ]:
def random_graph(nodes, edges):
    G = nx.Graph()
    G.add_nodes_from(np.arange(nodes))
    possible_edges = [(i, j) for i in range(nodes) for j in range(i + 1, nodes)]
    np.random.shuffle(possible_edges)
    G.add_edges_from(possible_edges[:edges])
    return G

In [ ]:
class Individ:
    def __init__(self, num_nodes):
        self.gene = np.random.randint(0,2,size=(num_nodes))

    def mutate(self, mutrate):
        mask = np.random.random(size=(len(self.gene),)) <= mutrate
        self.gene = np.logical_xor(self.gene, mask)

In [ ]:
def fitness(ind: Individ, G: nx.Graph) -> float:
    nodes = np.argwhere(ind.gene == 1)
    clique_edges = [
        (nodes[i][0], nodes[j][0])
        for i in range(len(nodes))
        for j in range(i + 1, len(nodes))
    ]
    edges_in_graph = [edge for edge in clique_edges if edge in G.edges()]
    return sum(ind.gene) - (len(clique_edges) - len(edges_in_graph))

In [ ]:
G = random_graph(10,20)
G.edges()

popsize = 100
selection = 20
mutrate = 0.01
max_gen = 1000

population = [Individ() for _ in range(popsize)]

generation = 0
while generation < max_gen:
    fittest = sorted(population, key=lambda x: fitness(x), reverse=True)
    fittest = fittest[0:selection]
    if fittest[0].fitness == len(nx.max_clique(G)):
        break
    # if generation % (max_gen*0.1) == 0:
    #     mutrate = mutrate * 0.5
    print(f"gen={generation}, mutrate={mutrate}, fittest={fittest[0].fitness}")
    #print(fittest[0].fitness)
    population = fittest[:]
    while len(population) < popsize:
        random.shuffle(fittest)
        mom = fittest.pop()
        dad = fittest.pop()
        child = Individ()
        for i in range(len(mom.gene)):
            child.gene[i] = mom.gene[i] if random.random() < 0.5 else dad.gene[i]
        child.mutate(mutrate)
        population.append(child)
        fittest.append(mom)
        fittest.append(dad)

    generation += 1

fittest = sorted(population, key=lambda x: x.fitness, reverse=True)
print(f"gen={generation}, mutrate={mutrate}, fittest={fittest[0].fitness}")